# Modell av en vargflock
Vi betraktar här en enkel modell av en vargpopulation i ett avgränsat geografiskt område. På grund av att populationen är liten, så måste man i modellen ta hänsyn till slumpmässigheten i när födslar och dödsfall inträffar. Det går alltså inte att ställa up en kontinuerlig modell av systemet. Födslar inträffar enligt en Poisson-process, där frekvensparametern $\lambda$ beror på antalet vargar, $N$,  enlig $\lambda=\lambda_0N$. Levetiden för en varg kan beskrivas med en normalfördelning med medelvärde (medellivslängd) 4 år och standardavvikelsen 1 år.

Modellen kan beskrivas med flödesschemat nedanför

![Flödesschema för vargmodellen](http://alfkjartan.github.io/images/varg_process_oriented.png "Populationsmodell")

In [1]:
%matplotlib
import simpy
import random
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt

Using matplotlib backend: Qt5Agg


In [37]:
def varg_generator_proc(env, lmbda0, pop, popsize):
    """ Varg-generator-processen. 
    
        Argument
        env     --  simuleringsmiljöobkektet
        lmbda0  --  medelfödselsfrekvensen per varg
        pop     --  lista innehållande varje levande varg
        popsize --  lista med observationer av populationsstorleker
    """
    k = 0
    while len(pop) > 0: 
        yield env.timeout( random.expovariate(lmbda0*len(pop)) )
        k += 1
        env.process( varg_proc(env, "Varg-%d" %k, pop, popsize) )
        
def varg_proc(env, namn, pop, popsize):
    """ Varg-processen
        Argument
        env     --  simuleringsmiljöobkektet
        namn    --  unik sträng för varj varg
        pop     --  lista innehållande varje levande varg
        popsize --  lista med observationer av populationsstorleker
    """

    pop.append(namn)
    popsize.append( (env.now, len(pop)) )
    yield env.timeout( max( (0, random.normalvariate(4, 1)) ) )
    pop.remove(namn)
    popsize.append( (env.now, len(pop)) )
    
def run_sim(initSize=8, lmbda0=0.25, horizon=20, plot=False):
    """ Körer simuleringsmodellen en gång. Returnerar storleken
        på populationen vid simuleringens slut.
    
        Argument
        initSize  -- storleken på populationen vid t=0
        lmbda0    -- medelfödselsfrekvensen per individ
        horizon   -- antal år att simulera
        plot      -- True om populationsstoleken ska plottas mot tiden
    """
    env = simpy.Environment()
    pop = []
    popsize = []
    
    for i in range(initSize):
        # Skapa 8 individer till att börja med
        env.process( varg_proc(env, "Ursprungsvarg-%d" %i, pop, popsize) )

    env.process( varg_generator_proc(env, lmbda0, pop, popsize) )
    env.run(until=horizon)

    if plot:
        popstorlek = np.asarray(popsize)
        plt.figure(1, figsize=(14,6))
        plt.step(popstorlek[:,0], popstorlek[:,1], where='post', 
                 linewidth=4 )
        plt.ylim((0,60))
        plt.xlim((0,21))
        plt.title("Populations storlek")
        
    return len(pop)

# Population at end of simulation
print( run_sim(plot=True) )
    

1


In [38]:
# Population at end of simulation
print( run_sim(plot=True) )

7


In [39]:
dir(random)

['BPF',
 'LOG4',
 'NV_MAGICCONST',
 'RECIP_BPF',
 'Random',
 'SG_MAGICCONST',
 'SystemRandom',
 'TWOPI',
 '_BuiltinMethodType',
 '_MethodType',
 '_Sequence',
 '_Set',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_acos',
 '_bisect',
 '_ceil',
 '_cos',
 '_e',
 '_exp',
 '_inst',
 '_itertools',
 '_log',
 '_pi',
 '_random',
 '_sha512',
 '_sin',
 '_sqrt',
 '_test',
 '_test_generator',
 '_urandom',
 '_warn',
 'betavariate',
 'choice',
 'choices',
 'expovariate',
 'gammavariate',
 'gauss',
 'getrandbits',
 'getstate',
 'lognormvariate',
 'normalvariate',
 'paretovariate',
 'randint',
 'random',
 'randrange',
 'sample',
 'seed',
 'setstate',
 'shuffle',
 'triangular',
 'uniform',
 'vonmisesvariate',
 'weibullvariate']